# 1. Search for region id's 
#### Use of Berlin Bezirke *get_all_regions()* and get Berlin's region id at the nuts1 - Level

In [ ]:
from datenguidepy.query_helper import get_all_regions
get_all_regions()
get_all_regions().query("name == 'Berlin' and level == 'nuts1'")

# 2. Search for names of statistics
#### Use *get_statistics()* to search for the variables you are interested in 

In [ ]:
from datenguidepy.query_helper import get_statistics

In [ ]:
descr = get_statistics()
descr[descr['short_description'].str.contains('Lebend Geborene', case = False)]

In [ ]:
descr[descr['short_description'].str.contains('Bevölkerungsstand', case = False)]

In [ ]:
descr[descr['short_description'].str.contains('Wohnfläche', case = False)]

# 3. Get data

#### Use *Query.region* to get data per Bezirk and append single dataframes

In [ ]:
from datenguidepy import Query
import pandas as pd
districts = get_all_regions().query("parent == '11' and level == 'nuts3'")

data_ana = pd.DataFrame()
for i, berlinid in enumerate(districts.index):
    query = Query.region(str(berlinid))
    query.add_field('BEVSTD')
    query.add_field('FLCNW1')
    query.add_field('BEV001')
    globals()[f"data_transformed_{str(i)}"] = query.results()
    temp = query.results()
    if data_ana.empty:
        data_ana = temp
    else:
        data_ana = data_ana.append(temp)
    

data_ana['AGS_G'] = data_ana['id'].astype('int64')

cols = ['id','name','year','BEV001','BEVSTD','FLCNW1','AGS_G']
data_ana = data_ana[cols]
data_ana.sort_index(inplace = True)

data_ana.head()

# 4. Get shapefile of Berlin Bezirke and join with dataframe
#### Use geopandas to open shapefile (downloaded from: https://data.technologiestiftung-berlin.de/dataset/bezirksgrenzen)


In [ ]:
import geopandas as gpd

de = gpd.read_file('ADD YOUR PATH HERE/Bezirksgrenzen/Bezirksgrenzen.shp')

de = de[['Land_schlu', 'Gemeinde_s', 'geometry']]
de['AGS_G']=(de['Land_schlu'].astype(int)*1000)+de['Gemeinde_s'].astype(int)
de['AGS_G']=de['AGS_G'].astype('int64')
de.sort_index(inplace = True)
de.head(4)

#### Some data preparation

In [ ]:
df = gpd.GeoDataFrame(de.merge(data_ana.loc[:100], on = 'AGS_G'))[['geometry', 'BEV001', 'name', 'year', 'BEVSTD', 'FLCNW1']]
df['births_per_inh'] = round(df['BEV001']/(df['BEVSTD']/1000),2)
df['sq_per_inh'] = round((df['FLCNW1']*1000)/df['BEVSTD'],2)
df.head(4)

# 5. Visualisation

In [ ]:
import param
import holoviews as hv
import panel as pn
import geoviews as gv
from bokeh.models import HoverTool
from cartopy import crs as ccrs 
from holoviews import opts
from bokeh.models import WMTSTileSource

gv.extension('bokeh')
pn.extension()

years = list(data_ana[data_ana['year']<2011].year.drop_duplicates().astype(int).sort_values())

varsv = ['births_per_inh', 'sq_per_inh']

data_ana['year'] = data_ana['year'].astype(int)

class BerlinStats(param.Parameterized):

    year = param.ObjectSelector(default=years[-1], objects=years)
    statistic =  param.ObjectSelector(default=varsv[-1], objects=varsv)
    
    @param.depends('year', 'statistic')
    def load_statistic(self):
        
        hover = (HoverTool(tooltips = [ ( 'Ort',   '@{name}'),
                                       ( 'year',   '@year'), 
                                       ( 'Births / 1.000 Inhabitants', '@births_per_inh{(0.00)}'), 
                                       ( 'Living space m2 / Inhabitant', '@sq_per_inh{(0.00)}'), 
                                      ]))
        p = (gv.Polygons(df[df['year']==self.year], 
                                    vdims=['births_per_inh', 'sq_per_inh', 'name', 'year'])
                                    .opts(color_index=self.statistic, colorbar=True, cmap='coolwarm'
                                          ,  tools=[hover], fill_alpha=.5))
        return p

bstats = BerlinStats()

base_map = gv.WMTS(WMTSTileSource(url='http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png'), 
                     crs=ccrs.PlateCarree())

stock_dmap = (base_map * gv.DynamicMap(bstats.load_statistic)).opts(width=600, height=600)

pn.Row(bstats.param, stock_dmap)